<font size="+4">Дашборд для [Дзена](https://zen.yandex.ru/)</font>

<font size="+2">Легенда и описание проекта</font>  
Почти всё время аналитика в [Дзене](https://zen.yandex.ru/) занимает анализ пользовательского взаимодействия с карточками статей.  
Каждую карточку статьи определяют её тема и источник (у него тоже есть тема). Примеры тем: «Красота и здоровье», «Россия», «Путешествия».  
Пользователей системы характеризует возрастная категория. Например, «26-30» или «45+».  

Каждую неделю начинающие менеджеры задают аналитику одни и те же вопросы по темам источников и карточек статей. Потому было решено сделать для менеджеров дашборд.  
Дашборд будет основываться на пайплайне, который будет брать данные из таблицы с сырыми данными, трансформировать данные и укладывать их в агрегирующую таблицу.

<font size="+2">Техническое задание</font>
- Бизнес-задача: анализ взаимодействия пользователей с карточками [Дзена](https://zen.yandex.ru/);
- На какие вопросы должен отвечать дашборд:
    - Сколько взаимодействий пользователей с карточками происходит в системе с разбивкой по темам карточек?
    - Как много карточек генерируют источники с разными темами?
    - Как соотносятся темы карточек и темы источников?
- Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
- Кто будет основным пользователем дашборда: менеджеры по анализу контента;
- Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек.
- По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа.
- Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек — абсолютные величины.
- Важность: все графики имеют равную важность;
- Источники данных для дашборда: дата-инженеры обещали подготовить агрегирующую таблицу dash_visits. Структура dash_visits:
    - `record_id` — первичный ключ;
    - `item_topic` — тема карточки;
    - `source_topic` — тема источника;
    - `age_segment` — возрастной сегмент;
    - `dt` — дата и время;
    - `visits` — количество событий.
- Таблица хранится в базе данных zen-db;
- Частота обновления данных: один раз в сутки, в полночь по UTC.

<font size="+2">Макет дашборда</font>  
![макет дашборда](https://3.downloader.disk.yandex.ru/preview/dcf8b286b175615aa7ebefe23ebad848200209a4112d5d6e2d5e11d85e548a15/inf/KVrkbHeJF4dufxAe_5IA5BYBsaFJwikallmI43omNCHscHtBCS8GGsVBnpET_VMqmZCKCB11klsJbbGBK5EIDg%3D%3D?uid=270737516&filename=Untitled_-_2020-07-06T160925.436_1594041010.png&disposition=inline&hash=&limit=0&content_type=image%2Fpng&owner_uid=270737516&tknv=v2&size=1200x1771)

<font size="+1">Ссылка на дашборд: https://public.tableau.com/app/profile/dmitriy.savelev/viz/_16912469370800/sheet4</font>  
<font size="+1">Ссылка на презентацию: https://disk.yandex.ru/d/tAovAZ42u2iOCA</font>

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': '---',  # имя пользователя
            'pwd': '---',    # пароль
            'host': '---',
            'port': '---',   # порт подключения
            'db': 'zen-db'}  # название базы данных

In [7]:
# Формируем строку соединения с БД
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'],
)

In [8]:
# Подключаемся к БД
engine = create_engine(connection_string)

In [9]:
# Формируем sql-запрос к БД
query = ''' SELECT *
            FROM dash_visits
        '''

In [10]:
# Выполняем запрос и сохраняем результат
# выполнения в DataFrame df
df = pd.io.sql.read_sql(query, con = engine, index_col='record_id') 

In [11]:
df.head(5) # Вывод последних 5 строк для первичного ознакомления с датасетом

,item_topic,source_topic,age_segment,dt,visits
record_id,,,,,
1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


In [12]:
# Проверка типов в выгруженных данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30745 entries, 1040597 to 1071341
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   item_topic    30745 non-null  object        
 1   source_topic  30745 non-null  object        
 2   age_segment   30745 non-null  object        
 3   dt            30745 non-null  datetime64[ns]
 4   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 1.4+ MB


Данные выгружены в корректны

In [13]:
# Сохранение датасета в файл dash_visits.csv

df.to_csv('dash_visits.csv')